# 📝 Énoncés des Exercices de Jointure SQL (Sakila)

In [2]:
# importer des bibliothèques
from sqlalchemy import create_engine, text
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

sns.set_theme(style="whitegrid")

# palette: noir et rouge
sns.set_palette(["red", "black", "gray"])

In [3]:
# path to file
DB_FILE = "sakila_master.db"   

# creer d'engine
engine = create_engine(f"sqlite:///{DB_FILE}", echo=False)
print(f"Connexion établie à la base de données : {DB_FILE}\n")

Connexion établie à la base de données : sakila_master.db



In [4]:
# Function pour faire une demande
def read_query_df(requete: str) -> pd.DataFrame:
    """
    Exécute une requête SQL et renvoie le résultat sous forme de DataFrame.

    Args:
        requete (str): La requête SQL à exécuter.

    Returns:
        pd.DataFrame: Le DataFrame contenant les données résultantes.

    Raises:
        Exception: Une erreur est affichée si la récupération ou l'exécution
            de la requête échoue.
    """

    try:
        with engine.connect() as conn:
            df = pd.read_sql(text(requete), conn)

        return df

    except Exception as e:
        print(f"Une erreur s'est produite dans la récupération des données...")

### Exercice 1 : Jointure Simple à 2 Tables

**Objectif :** Joindre la table des clients à la table des locations pour trouver les clients les plus actifs récemment.

| Tables Impliquées | Clauses Spécifiques |
| :--- | :--- |
| `customer` et `rental` | Alias, `ORDER BY`, `LIMIT` |

**Énoncé :**

Écrivez une requête SQL pour lister les **10 clients** (`first_name`, `last_name` de la table `customer`) ayant effectué leur location la **plus récente**. Affichez également la date de la location (`rental_date` de la table `rental`).

* **Instructions :**
    * Utilisez des **alias** clairs pour les tables (`c` pour `customer`, `r` pour `rental`).
    * Triez le résultat par la date de location la plus récente (`ORDER BY`).
    * Limitez l'affichage aux 10 premiers résultats (`LIMIT`).


In [5]:
# Exersice 1
req = """
SELECT C.customer_id AS id, C.last_name AS name, C.first_name AS f_name, R.rental_date AS date
FROM rental AS R
JOIN customer as C
ON C.customer_id = R.customer_id
GROUP BY C.customer_id
ORDER BY R.rental_date DESC
LIMIT 10
"""

df = read_query_df(req)
print(df)



    id        name   f_name                     date
0  195        SIMS  VANESSA  2005-07-07 13:01:35.000
1  226      LITTLE  MAUREEN  2005-06-21 16:58:50.000
2  335     MAULDIN  GREGORY  2005-06-21 09:20:31.000
3  555    LOMBARDI   DWIGHT  2005-06-21 02:30:37.000
4  487  POINDEXTER   HECTOR  2005-06-20 16:47:57.000
5  598    DELVALLE     WADE  2005-06-20 10:10:29.000
6  453      MARTEL   CALVIN  2005-06-19 23:08:50.000
7  418        EAST     JEFF  2005-06-19 20:32:19.000
8  567     MCADAMS  ALFREDO  2005-06-19 12:58:53.000
9  458        DOWD    LLOYD  2005-06-19 08:42:12.000


### Exercice 2 : Jointure à 3 Tables

**Objectif :** Relier la transaction (`rental`) au contexte du produit (`film`) via la table d'inventaire.

| Tables Impliquées | Clauses Spécifiques |
| :--- | :--- |
| `rental`, `inventory` et `film` | Alias, `ORDER BY`, `LIMIT`, `WHERE` |

**Énoncé :**inventory

Quel est le **titre du film** (`title` de la table `film`) et la **date de location** (`rental_date` de la table `rental`) des **5 dernières locations** ?

* **Filtre :** Incluez uniquement les films dont la classification (`rating`) est `'PG-13'`.
* **Instructions :**
    * Triez par la date de location la plus récente (`ORDER BY`).
    * Limitez l'affichage aux 5 premiers résultats (`LIMIT`).


In [6]:
# Exersice 2: join 3 tableaux
req = """
SELECT R.rental_date AS date, I.film_id as film_id, F.title AS title
FROM rental AS R
JOIN inventory as I
ON R.inventory_id = I.inventory_id
JOIN film As F
ON I.film_id = F.film_id
ORDER BY R.rental_date
LIMIT 5
"""

df = read_query_df(req)
df

,date,film_id,title
0,2005-05-24 22:53:30.000,80,BLANKET BEVERLY
1,2005-05-24 22:54:33.000,333,FREAKY POCUS
2,2005-05-24 23:03:39.000,373,GRADUATE LORD
3,2005-05-24 23:04:41.000,535,LOVE SUICIDES
4,2005-05-24 23:05:21.000,450,IDOLS SNATCHERS


### Exercice 3 : Jointure à 4 Tables (Liaison Many-to-Many)

**Objectif :** Traverser une table de liaison pour relier des entités et ajouter une dimension (langue).

| Tables Impliquées | Clauses Spécifiques |
| :--- | :--- |
| `actor`, `film_actor`, `film` et `language` | Alias, `LIMIT`, `WHERE`, `DISTINCT` |

**Énoncé :**

Listez les **15 premiers acteurs** (`first_name`, `last_name` de la table `actor`) qui ont joué dans des films dont la **langue originale** (`name` de la table `language`) est l'**Anglais** (`'English'`). Affichez uniquement le nom et le prénom de l'acteur.

* **Instructions :**
    * Utilisez **`DISTINCT`** pour garantir que chaque acteur n'apparaisse qu'une seule fois.
    * Triez par le nom de famille de l'acteur (`ORDER BY`).
    * Limitez le résultat à 15 entrées (`LIMIT`).


In [34]:
list_table = ['actor', 'film_actor', 'film', 'language']

def show_columns(list_table:list):
    for i in list_table:
        req = f"SELECT * FROM {i};"
        print({i}, read_query_df(req).columns)

show_columns(list_table)

{'actor'} Index(['actor_id', 'first_name', 'last_name', 'last_update'], dtype='object')
{'film_actor'} Index(['actor_id', 'film_id', 'last_update'], dtype='object')
{'film'} Index(['film_id', 'title', 'description', 'release_year', 'language_id',
       'original_language_id', 'rental_duration', 'rental_rate', 'length',
       'replacement_cost', 'rating', 'special_features', 'last_update'],
      dtype='object')
{'language'} Index(['language_id', 'name', 'last_update'], dtype='object')


In [33]:
# Exercice 3 : Jointure à 4 Tables (Liaison Many-to-Many)
req = """
SELECT DISTINCT A.first_name, A.last_name
FROM actor AS A
JOIN film_actor as F_A
ON A.actor_id = F_A.actor_id
JOIN film as F
ON F_A.film_id = F.film_id
JOIN language as L
ON F.language_id = L.language_id
WHERE L.name = 'English'
ORDER BY A.last_name 
LIMIT 15;
"""

df = read_query_df(req)
df


,first_name,last_name
0,CHRISTIAN,AKROYD
1,KIRSTEN,AKROYD
2,DEBBIE,AKROYD
3,CUBA,ALLEN
4,KIM,ALLEN
5,MERYL,ALLEN
6,ANGELINA,ASTAIRE
7,RUSSELL,BACALL
8,JESSICA,BAILEY
9,AUDREY,BAILEY


### Exercice 4 : Jointure à 5 Tables (Transaction et Structure)

**Objectif :** Intégrer la dimension humaine (`staff`) et structurelle (`store`) à la transaction (`rental`) et au produit (`film`).

| Tables Impliquées | Clauses Spécifiques |
| :--- | :--- |
| `rental`, `inventory`, `film`, `store` et `staff` | Alias, `ORDER BY`, `LIMIT`, `WHERE` |

**Énoncé :**

Pour le **magasin n°2** (`store_id = 2`), listez les **5 derniers films loués** (`title` de la table `film`), la **date de location** (`rental_date` de la table `rental`) et le **prénom/nom de l'employé** (`first_name`, `last_name` de la table `staff`) qui a géré la transaction.

* **Instructions :**
    * Utilisez la clause `WHERE` pour filtrer sur le magasin n°2.
    * Triez par la date de location la plus récente (`ORDER BY`).
    * Limitez l'affichage aux 5 résultats (`LIMIT`).

In [36]:
list_table = ["rental", "inventory", "film","store", "staff"]
show_columns(list_table)

{'rental'} Index(['rental_id', 'rental_date', 'inventory_id', 'customer_id',
       'return_date', 'staff_id', 'last_update'],
      dtype='object')
{'inventory'} Index(['inventory_id', 'film_id', 'store_id', 'last_update'], dtype='object')
{'film'} Index(['film_id', 'title', 'description', 'release_year', 'language_id',
       'original_language_id', 'rental_duration', 'rental_rate', 'length',
       'replacement_cost', 'rating', 'special_features', 'last_update'],
      dtype='object')
{'store'} Index(['store_id', 'manager_staff_id', 'address_id', 'last_update'], dtype='object')
{'staff'} Index(['staff_id', 'first_name', 'last_name', 'address_id', 'picture', 'email',
       'store_id', 'active', 'username', 'password', 'last_update'],
      dtype='object')
